In [81]:
import json
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from tqdm import tqdm
import os

splits = {'train': 'train.json', 'validation': 'validation.json'}
train_data = pd.read_json("hf://datasets/sander-wood/irishman/" + splits["train"])
val_data = pd.read_json("hf://datasets/sander-wood/irishman/" + splits["validation"])

train_data
val_data

,control code,abc notation
0,S:2\nB:9\nE:3\nB:10\n,X:1\nL:1/8\nM:6/8\nK:Bb\n F | B2 d c2 f | edc ...
1,S:2\nB:8\nE:3\nB:8\n,X:2\nL:1/8\nM:4/4\nK:G\n GBAB G2 ge | dBGB AE ...
2,S:5\nB:1\nE:1\nB:5\nE:1\nE:5\nB:5\nE:0\nE:4\nE...,X:3\nL:1/8\nM:4/4\nK:A\n d |: cABG A2 ED | CEA...
3,S:2\nB:8\nE:6\nB:8\n,X:4\nL:1/8\nM:2/4\nK:Amin\n|: ce/d/ cB | Aa^ga...
4,S:2\nB:8\nE:6\nB:8\n,X:5\nL:1/8\nM:4/4\nK:Amix\n cA A2 EA A2 | BGdG...
...,...,...
2157,S:2\nB:9\nE:5\nB:9\n,X:2158\nL:1/8\nM:6/8\nK:G\n f | edB BAB | GEF ...
2158,S:1\nB:16\n,X:2159\nL:1/8\nQ:1/8=232\nM:4/4\nK:D\n|: A2 AB...
2159,S:2\nB:9\nE:5\nB:9\n,"X:2160\nL:1/8\nM:6/8\nK:Amix\n F2 D D=CA, | =C..."
2160,S:1\nB:16\n,X:2161\nL:1/8\nM:4/4\nK:G\n g3 d BGAB | cBAG F...


In [82]:
!pip install tensorboard


In [83]:
train_data


,control code,abc notation
0,S:2\nB:5\nE:5\nB:6\n,X:1\nL:1/8\nM:4/4\nK:Emin\n|: E2 EF E2 EF | DE...
1,S:1\nB:25\n,X:2\nL:1/4\nM:3/4\nK:C\n G | E3/2 E/ E | G2 G ...
2,S:2\nB:9\nE:7\nB:9\n,X:3\nL:1/8\nQ:1/4=100\nM:2/4\nK:Emin\n E | ABc...
3,S:4\nB:1\nE:1\nB:8\nE:3\nE:1\nB:1\nE:1\nE:4\nE...,"X:4\nL:1/8\nM:6/8\nK:A\n|: EFG ||""A"" A2 A A2 A..."
4,S:2\nB:8\nE:5\nB:8\n,X:5\nL:1/8\nM:6/8\nK:G\n (G/A/B).D D>ED | DcB ...
...,...,...
214117,S:1\nB:17\n,X:214118\nL:1/8\nM:2/4\nK:F\n C | FFFF | AFFF ...
214118,S:2\nB:9\nE:6\nB:9\n,X:214119\nL:1/8\nM:4/4\nK:G\n (g/>a/) | (b<g) ...
214119,S:3\nB:8\nE:3\nB:4\nE:5\nE:4\nB:6\n,X:214120\nL:1/8\nM:3/4\nK:A\n|: [Eca]3 c/a/ B3...
214120,S:3\nB:8\nE:5\nB:8\nE:2\nE:3\nB:16\n,"X:214121\nL:1/8\nQ:1/4=110\nM:2/4\nK:D\n""_See ..."


In [84]:
val_data

,control code,abc notation
0,S:2\nB:9\nE:3\nB:10\n,X:1\nL:1/8\nM:6/8\nK:Bb\n F | B2 d c2 f | edc ...
1,S:2\nB:8\nE:3\nB:8\n,X:2\nL:1/8\nM:4/4\nK:G\n GBAB G2 ge | dBGB AE ...
2,S:5\nB:1\nE:1\nB:5\nE:1\nE:5\nB:5\nE:0\nE:4\nE...,X:3\nL:1/8\nM:4/4\nK:A\n d |: cABG A2 ED | CEA...
3,S:2\nB:8\nE:6\nB:8\n,X:4\nL:1/8\nM:2/4\nK:Amin\n|: ce/d/ cB | Aa^ga...
4,S:2\nB:8\nE:6\nB:8\n,X:5\nL:1/8\nM:4/4\nK:Amix\n cA A2 EA A2 | BGdG...
...,...,...
2157,S:2\nB:9\nE:5\nB:9\n,X:2158\nL:1/8\nM:6/8\nK:G\n f | edB BAB | GEF ...
2158,S:1\nB:16\n,X:2159\nL:1/8\nQ:1/8=232\nM:4/4\nK:D\n|: A2 AB...
2159,S:2\nB:9\nE:5\nB:9\n,"X:2160\nL:1/8\nM:6/8\nK:Amix\n F2 D D=CA, | =C..."
2160,S:1\nB:16\n,X:2161\nL:1/8\nM:4/4\nK:G\n g3 d BGAB | cBAG F...


In [85]:
print("Nombre de chansons dans le train :", len(train_data))
print("Nombre de chansons dans la validation :", len(val_data))


Nombre de chansons dans le train : 214122
Nombre de chansons dans la validation : 2162


Affichez la première chanson du fichier d&#39;entraînement. Que pouvez-vous en déduire sur sa structure ?

Chaque partition est un texte : on peut la considérer comme une séquence de caractères.

RNN / LSTM va apprendre à prédire le caractère suivant, donc il va apprendre les notes, rythmes et motifs.

Les caractères uniques seront : lettres (A-G, a-g), chiffres (1-3…), symboles (|, :, ^, etc.).

Les lignes d’en-tête (X:, L:, M:, K:) fournissent du contexte musical.

In [86]:
first_song = train_data.loc[0, 'abc notation']
print("Première chanson :\n", first_song)


Première chanson :
 X:1
L:1/8
M:4/4
K:Emin
|: E2 EF E2 EF | DEFG AFDF | E2 EF E2 B2 |1 efe^d e2 e2 :|2 efe^d e3 B |: e2 ef g2 fe | 
 defg afdf |1 e2 ef g2 fe | efe^d e3 B :|2 g2 bg f2 af | efe^d e2 e2 ||


In [87]:
# Concaténer toutes les partitions en une seule grande chaîne
all_text = "".join(train_data['abc notation'].tolist())

# Trouver les caractères uniques
unique_chars = sorted(list(set(all_text)))

print("Caractères uniques :", unique_chars)
print("Nombre de caractères uniques :", len(unique_chars))


Caractères uniques : ['\n', ' ', '!', '"', '#', '$', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~']
Nombre de caractères uniques : 95


Raisons principales d’utiliser des indices

Compatibilité avec les tenseurs : PyTorch et TensorFlow ne traitent que des nombres.

Facilité de batch processing : on peut créer des matrices (batch_size × sequence_length) pour entraîner le RNN.

Embeddings : les indices permettent de projeter chaque caractère dans un espace vectoriel dense.

Génération : le RNN prédit des probabilités sur les indices, et on reconvertit les indices en caractères pour lire la partition.

Écrivez un dictionnaire permettant de passer d&#39;un caractère à un index.

In [88]:
# char2idx : chaque caractère unique a un index unique
char2idx = {ch: idx for idx, ch in enumerate(unique_chars)}

# Exemple d'utilisation
print("Index du caractère 'E' :", char2idx['E'])


Index du caractère 'E' : 37


Écrivez une liste permettant de passer d&#39;un index à un caractère.

In [89]:
# Option 1 : avec un dictionnaire
idx2char = {idx: ch for idx, ch in enumerate(unique_chars)}

# Option 2 : plus simple avec une liste
idx2char_list = list(unique_chars)

# Tester
print("Caractère pour l'index 30 (dictionnaire) :", idx2char[37])
print("Caractère pour l'index 30 (liste) :", idx2char_list[37])


Caractère pour l'index 30 (dictionnaire) : E
Caractère pour l'index 30 (liste) : E


On va créer une fonction vectorize_string qui prend une chaîne (partition ABC) et la transforme en liste d’indices grâce au dictionnaire char2idx que tu as créé.

In [90]:
def vectorize_string(text, char2idx):
    """
    Transforme une chaîne de caractères en une liste d'indices.

    Args:
        text (str): la chaîne de caractères à vectoriser
        char2idx (dict): dictionnaire caractère → index

    Returns:
        List[int]: liste d'indices correspondant aux caractères
    """
    return [char2idx[ch] for ch in text]

# Test avec la première partition du dataset
first_song = train_data['abc notation'].iloc[0]
vectorized_first_song = vectorize_string(first_song, char2idx)

# Afficher un petit extrait
print("Premiers indices :", vectorized_first_song[:20])
print("Longueur de la partition :", len(vectorized_first_song))


Premiers indices : [56, 26, 17, 0, 44, 26, 17, 15, 24, 0, 45, 26, 20, 15, 20, 0, 43, 26, 37, 77]
Longueur de la partition : 183


On va maintenant gérer le padding pour que toutes les séquences aient la même longueur et puissent former des batches pour le RNN.

In [91]:
#Trouver la longueur maximale des partitions
# Longueurs de toutes les partitions
lengths = train_data['abc notation'].apply(len)

# Longueur maximale
max_length = lengths.max()
print("Longueur maximale des partitions :", max_length)


Longueur maximale des partitions : 2968


Fonction pour padding ou troncature


len(text) → longueur actuelle de la partition.

max_length - len(text) → nombre d’espaces à ajouter si la partition est trop courte.

Si la partition est trop longue, on la tronque pour ne garder que max_length caractères.

In [92]:
def pad_or_truncate(text, max_length):
    """
    Ajuste la chaîne à une longueur fixe.
    - Si la chaîne est plus courte, ajoute des espaces à la fin.
    - Si la chaîne est plus longue, coupe la fin.

    Args:
        text (str): chaîne à ajuster
        max_length (int): longueur désirée

    Returns:
        str: chaîne ajustée
    """
    if len(text) < max_length:
        # Ajouter des espaces
        return text + ' ' * (max_length - len(text))
    else:
        # Couper la fin
        return text[:max_length]

# Test avec la première partition
padded_first_song = pad_or_truncate(first_song, max_length)
print("Longueur après padding :", len(padded_first_song))


Longueur après padding : 2968


Nous voulons rassembler tout le prétraitement en une fonction qui retourne les mappings et les
données vectorisées prêtes à l&#39;emploi.

In [93]:
def preprocess_dataset(dataset):
    """
    Prétraite le dataset pour le RNN.
    - Crée char2idx et idx2char
    - Trouve la longueur maximale
    - Applique padding et vectorise les partitions

    Args:
        dataset (pd.DataFrame): DataFrame avec la colonne 'abc notation'

    Returns:
        char2idx (dict)
        idx2char (dict)
        vectorized_data (list of list of int)
        max_length (int)
    """
    # Concaténer toutes les partitions pour trouver les caractères uniques
    all_text = ''.join(dataset['abc notation'].tolist())
    unique_chars = sorted(list(set(all_text)))

    # Mappings
    char2idx = {ch: idx for idx, ch in enumerate(unique_chars)}
    idx2char = {idx: ch for idx, ch in enumerate(unique_chars)}

    # Longueur maximale
    max_length = max(dataset['abc notation'].apply(len))

    # Vectorisation + padding
    vectorized_data = []
    for song in dataset['abc notation']:
        # Padding ou troncature
        if len(song) < max_length:
            song = song + ' ' * (max_length - len(song))
        else:
            song = song[:max_length]
        # Convertir en indices
        vectorized_data.append([char2idx[ch] for ch in song])

    return char2idx, idx2char, vectorized_data, max_length

# Exemple d'utilisation
char2idx, idx2char, train_vectorized, max_len = preprocess_dataset(train_data)

print("Nombre de caractères uniques :", len(char2idx))
print("Premiers indices de la première partition :", train_vectorized[0][:20])
print("Longueur maximale :", max_len)


Nombre de caractères uniques : 95
Premiers indices de la première partition : [56, 26, 17, 0, 44, 26, 17, 15, 24, 0, 45, 26, 20, 15, 20, 0, 43, 26, 37, 77]
Longueur maximale : 2968


On va maintenant créer la classe MusicDataset et préparer les DataLoaders pour le RNN.


Pourquoi créer un Dataset PyTorch ?

PyTorch fonctionne avec torch.utils.data.Dataset et DataLoader pour gérer :

Les batches (groupes d’exemples)

Le shuffle (mélange des données à chaque epoch)

Le prétraitement à la volée (par exemple vectorisation ou padding si nécessaire)

Donc même si tu as déjà tes partitions vectorisées, il faut mettre ces données dans un format que PyTorch comprend : c’est exactement ce que fait MusicDataset.

In [94]:
#Classe MusicDataset
class MusicDataset(Dataset):
    def __init__(self, vectorized_data):
        """
        Args:
            vectorized_data (list of list of int): partitions vectorisées avec padding
        """
        self.data = vectorized_data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        seq = self.data[idx]
        # Séquence d'entrée : tout sauf le dernier caractère
        x = torch.tensor(seq[:-1], dtype=torch.long)
        # Séquence cible : tout sauf le premier caractère (décalée d'un pas)
        y = torch.tensor(seq[1:], dtype=torch.long)
        return x, y


C’est quoi un DataLoader ?

Un DataLoader est un outil PyTorch qui sert à :

donner les données au modèle pendant l’entraînement,

par petits paquets (batches),
dans le bon format,
automatiquement.

Tu peux le voir comme un serveur de données pour ton modèle.

In [95]:
#Initialiser les DataLoaders
from torch.utils.data import DataLoader

batch_size = 8

# Dataset
train_dataset = MusicDataset(train_vectorized)
# Pour validation, si tu as déjà vectorisé val_data :
# val_char2idx, val_idx2char, val_vectorized, _ = preprocess_dataset(val_data)
# val_dataset = MusicDataset(val_vectorized)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Vérification d'un batch
x_batch, y_batch = next(iter(train_loader))
print("Shape x_batch :", x_batch.shape)  # (batch_size, sequence_length)
print("Shape y_batch :", y_batch.shape)
print("Premier batch x :", x_batch[0][:20])
print("Premier batch y :", y_batch[0][:20])


Shape x_batch : torch.Size([8, 2967])
Shape y_batch : torch.Size([8, 2967])
Premier batch x : tensor([56, 26, 17, 21, 20, 16, 24, 17,  0, 44, 26, 17, 15, 24,  0, 49, 26, 17,
        15, 20])
Premier batch y : tensor([26, 17, 21, 20, 16, 24, 17,  0, 44, 26, 17, 15, 24,  0, 49, 26, 17, 15,
        20, 29])


In [96]:
class MusicRNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size):
        super(MusicRNN, self).__init__()

        # 1️⃣ Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # 2️⃣ LSTM
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_size,
            batch_first=True
        )

        # 3️⃣ Couche dense pour la prédiction
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        """
        x : (batch_size, sequence_length)
        """
        # Embedding
        x = self.embedding(x)
        # x : (batch_size, sequence_length, embedding_dim)

        # LSTM
        out, _ = self.lstm(x)
        # out : (batch_size, sequence_length, hidden_size)

        # Projection vers le vocabulaire
        out = self.fc(out)
        # out : (batch_size, sequence_length, vocab_size)

        return out


In [97]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

criterion = nn.CrossEntropyLoss()


In [98]:
# DEBUG MODE — ONLY A FEW SONGS
train_data = train_data.iloc[:10000]
val_data   = val_data.iloc[:1000]


In [99]:
# Concatenate all training songs
all_text = "".join(train_data["abc notation"])

unique_chars = sorted(list(set(all_text)))
vocab_size = len(unique_chars)

char2idx = {ch: i for i, ch in enumerate(unique_chars)}
idx2char = {i: ch for ch, i in char2idx.items()}

print("Vocab size:", vocab_size)


Vocab size: 92


In [100]:
max_len = 100

def vectorize_with_existing_vocab(dataset, char2idx, max_len):
    vectorized = []

    for song in dataset["abc notation"]:
        if len(song) < max_len:
            song = song + " " * (max_len - len(song))
        else:
            song = song[:max_len]

        vectorized.append([char2idx[ch] for ch in song])

    return vectorized


train_vectorized = vectorize_with_existing_vocab(train_data, char2idx, max_len)
val_vectorized   = vectorize_with_existing_vocab(val_data, char2idx, max_len)


In [101]:
class MusicDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        seq = torch.tensor(self.data[idx], dtype=torch.long)
        return seq[:-1], seq[1:]


In [102]:
batch_size = 8

train_dataset = MusicDataset(train_vectorized)
val_dataset   = MusicDataset(val_vectorized)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [103]:
class MusicRNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        out, _ = self.lstm(x)
        out = self.fc(out)
        return out


In [104]:
num_training_iterations = 25
embedding_dim = 16
hidden_size = 64
learning_rate = 1e-3


In [105]:
def train_model(
    model,
    train_loader,
    val_loader,
    num_iterations,
    learning_rate
):
    print("🚀 Training started")
    model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    train_losses = []
    val_losses = []

    for epoch in range(num_iterations):
        # ===== TRAIN =====
        model.train()
        train_loss = 0

        for x, y in train_loader:
            x, y = x.to(device), y.to(device)

            optimizer.zero_grad()
            outputs = model(x)

            loss = criterion(
                outputs.view(-1, outputs.size(-1)),
                y.view(-1)
            )

            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_loss /= len(train_loader)
        train_losses.append(train_loss)

        # ===== VALIDATION =====
        model.eval()
        val_loss = 0

        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)

                outputs = model(x)
                loss = criterion(
                    outputs.view(-1, outputs.size(-1)),
                    y.view(-1)
                )

                val_loss += loss.item()

        val_loss /= len(val_loader)
        val_losses.append(val_loss)

        print(
            f"Epoch {epoch+1}/{num_iterations} | "
            f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}"
        )

    return train_losses, val_losses


In [106]:
model = MusicRNN(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    hidden_size=hidden_size
).to(device)

train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    num_iterations=num_training_iterations,
    learning_rate=learning_rate
)


🚀 Training started
Epoch 1/25 | Train Loss: 2.0219 | Val Loss: 1.6045
Epoch 2/25 | Train Loss: 1.4919 | Val Loss: 1.4409
Epoch 3/25 | Train Loss: 1.3751 | Val Loss: 1.3490
Epoch 4/25 | Train Loss: 1.3060 | Val Loss: 1.3057
Epoch 5/25 | Train Loss: 1.2618 | Val Loss: 1.2606
Epoch 6/25 | Train Loss: 1.2302 | Val Loss: 1.2436
Epoch 7/25 | Train Loss: 1.2072 | Val Loss: 1.2195
Epoch 8/25 | Train Loss: 1.1899 | Val Loss: 1.2067
Epoch 9/25 | Train Loss: 1.1760 | Val Loss: 1.1876
Epoch 10/25 | Train Loss: 1.1645 | Val Loss: 1.1838
Epoch 11/25 | Train Loss: 1.1545 | Val Loss: 1.1748
Epoch 12/25 | Train Loss: 1.1462 | Val Loss: 1.1716
Epoch 13/25 | Train Loss: 1.1394 | Val Loss: 1.1653
Epoch 14/25 | Train Loss: 1.1327 | Val Loss: 1.1641
Epoch 15/25 | Train Loss: 1.1271 | Val Loss: 1.1546
Epoch 16/25 | Train Loss: 1.1231 | Val Loss: 1.1538
Epoch 17/25 | Train Loss: 1.1174 | Val Loss: 1.1511
Epoch 18/25 | Train Loss: 1.1130 | Val Loss: 1.1438
Epoch 19/25 | Train Loss: 1.1087 | Val Loss: 1.1394
Ep

([2.021895997810364,
  1.491889596748352,
  1.375066372680664,
  1.3060491904258729,
  1.2617574676513672,
  1.2301690299034118,
  1.20723211312294,
  1.1898989569664002,
  1.1760053245544433,
  1.164456897354126,
  1.1545481276988983,
  1.146204981279373,
  1.1393767098903655,
  1.1327054129123688,
  1.127134585428238,
  1.1231373793125152,
  1.117380269908905,
  1.1130458740234375,
  1.1087248457431793,
  1.1050572604179383,
  1.1016377069473267,
  1.0983255541801453,
  1.095451682806015,
  1.0919370095252992,
  1.089840336418152],
 [1.6045499057769776,
  1.4408847160339355,
  1.3489945907592773,
  1.305672607421875,
  1.2605695257186889,
  1.243556378364563,
  1.2194760570526122,
  1.2067387952804565,
  1.1875833959579467,
  1.1837646899223329,
  1.1747944254875182,
  1.1715618534088135,
  1.1653241157531737,
  1.1640623340606688,
  1.1545880661010741,
  1.153799864768982,
  1.1510713152885437,
  1.1437648038864137,
  1.1393994073867797,
  1.137637062549591,
  1.1367420415878295,
  

In [111]:
def generate_music(model, start_sequence, char2idx, idx2char, length=200, device='cpu', sample=False, temperature=1.0):
    model.eval()
    input_seq = [char2idx[ch] for ch in start_sequence]
    input_tensor = torch.tensor(input_seq, dtype=torch.long).unsqueeze(0).to(device)

    generated = start_sequence

    hidden = None  # pour LSTM, on peut passer None pour commencer

    for _ in range(length):
        outputs = model(input_tensor)  # shape: [1, seq_len, vocab_size]
        last_logits = outputs[0, -1]  # on prend le dernier caractère

        if sample:
            probs = torch.softmax(last_logits / temperature, dim=-1)
            next_idx = torch.multinomial(probs, num_samples=1).item()
        else:
            next_idx = torch.argmax(last_logits).item()

        next_char = idx2char[next_idx]
        generated += next_char

        # mise à jour de input_tensor pour le prochain pas
        next_input = torch.tensor([[next_idx]], dtype=torch.long).to(device)
        input_tensor = torch.cat([input_tensor, next_input], dim=1)

    return generated


In [112]:
start_sequence = "X:1\nT:MySong\nM:4/4\nK:C\n"

# génération greedy
song_greedy = generate_music(model, start_sequence, char2idx, idx2char, length=200, device=device, sample=False)
print("🎵 Generated Song (Greedy):\n", song_greedy)

# génération avec échantillonnage
song_sampled = generate_music(model, start_sequence, char2idx, idx2char, length=200, device=device, sample=True, temperature=1.2)
print("🎵 Generated Song (Sampled):\n", song_sampled)


🎵 Generated Song (Greedy):
 X:1
T:MySong
M:4/4
K:C
 c2 | c2 c2 c2 c2 c2 | c2 c2 c2 c2 c2 | c2 c2 c2 c2 c2 | c2 c2 c2 c2 c2 | c2 c2 c2 c2 c2 | c2 c2 c2 c2 c2 | c2 c2 c2 c2 c2 | c2 c2 c2 c2 c2 | c2 c2 c2 c2 c2 | c2 c2 c2 c2 c2 | c2 c2 c2 c2 c2 | c2 c2 c
🎵 Generated Song (Sampled):
 X:1
T:MySong
M:4/4
K:C
 F4 F2 E2 A2 |[QOMK2 edcd c2 e4 c2 | BfdcBA A2 cBc | Acd4 a2 e2 | e4 a2ga^g2 | a4 f2af ecBE | d3 B A2A2 d2B2 | cdc2 Bdfedefe | fdedfe edcB ABcA |12 =G3 E4 DFGF/F/ | B4 A2A2B2Ad | efeg e2.B2-Bc :: B2fe
